In [1]:
import xgboost as xgb
print(xgb.__version__)

3.1.1


In [2]:
import sys, xgboost as xgb
print(sys.executable)        # should point to .../.venv/bin/python
print(xgb.__version__)       # should print 3.0.4
print(xgb.__file__)          # should live under .../.venv/...

/home/tamer/anaconda3/envs/end2end/bin/python
3.1.1
/home/tamer/anaconda3/envs/end2end/lib/python3.11/site-packages/xgboost/__init__.py


In [3]:
# ==============================================
# 1. Imports
# ==============================================
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import optuna
import mlflow
import mlflow.xgboost

/home/tamer/anaconda3/envs/end2end/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# ==============================================
# 2. Load processed datasets
# ==============================================
train_df = pd.read_csv("../data/processed/feature_engineered_train.csv")
eval_df  = pd.read_csv("../data/processed/feature_engineered_eval.csv")


# Define target + features
target = "price"
X_train, y_train = train_df.drop(columns=[target]), train_df[target]
X_eval, y_eval   = eval_df.drop(columns=[target]), eval_df[target]

print("Train shape:", X_train.shape)
print("Eval shape:", X_eval.shape)

Train shape: (576815, 39)
Eval shape: (148448, 39)


In [5]:
# ==============================================
# 3. Define Optuna objective function with MLflow
# ==============================================
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1,
        "tree_method": "hist",
    }

    with mlflow.start_run(nested=True):
        model = XGBRegressor(**params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_eval)
        rmse = float(np.sqrt(mean_squared_error(y_eval, y_pred)))
        mae = float(mean_absolute_error(y_eval, y_pred))
        r2 = float(r2_score(y_eval, y_pred))

        # Log hyperparameters + metrics
        mlflow.log_params(params)
        mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

    return rmse

In [6]:
# ==============================================
# 4. Run Optuna study with MLflow
# ==============================================
# Force MLflow to always use the root project mlruns folder
mlflow.set_tracking_uri("../mlruns")
mlflow.set_experiment("xgboost_optuna_housing")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)

print("Best params:", study.best_trial.params)

/home/tamer/anaconda3/envs/end2end/lib/python3.11/site-packages/mlflow/tracking/_tracking_service/utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)
2025/11/08 20:20:29 INFO mlflow.tracking.fluent: Experiment with name 'xgboost_optuna_housing' does not exist. Creating a new experiment.
[I 2025-11-08 20:20:29,514] A new study created in memory with name: no-name-0314fd24-fb22-449f-b544-d2846b445eab


[I 2025-11-08 20:20:48,421] Trial 0 finished with value: 80454.39488404279 and parameters: {'n_estimators': 523, 'max_depth': 3, 'learning_rate': 0.07455890814673258, 'subsample': 0.8327463948517818, 'colsample_bytree': 0.9973405703859128, 'min_child_weight': 7, 'gamma': 4.910282139461838, 'reg_alpha': 2.98968817959368, 'reg_lambda': 0.040223266762575524}. Best is trial 0 with value: 80454.39488404279.
[I 2025-11-08 20:21:17,951] Trial 1 finished with value: 72501.23841528659 and parameters: {'n_estimators': 306, 'max_depth': 10, 'learning_rate': 0.07617138219692937, 'subsample': 0.9379012985594393, 'colsample_bytree': 0.806821703627337, 'min_child_weight': 7, 'gamma': 2.358846843437264, 'reg_alpha': 1.025363673191934e-07, 'reg_lambda': 8.322332815607044}. Best is trial 1 with value: 72501.23841528659.
[I 2025-11-08 20:21:59,424] Trial 2 finished with value: 70478.1188318549 and parameters: {'n_estimators': 972, 'max_depth': 5, 'learning_rate': 0.05889527650472125, 'subsample': 0.52506

Best params: {'n_estimators': 249, 'max_depth': 7, 'learning_rate': 0.1511582578380317, 'subsample': 0.9822836498176734, 'colsample_bytree': 0.5001811357376785, 'min_child_weight': 4, 'gamma': 0.004054170311807104, 'reg_alpha': 2.66059949602335e-06, 'reg_lambda': 7.688359542523061}


In [7]:
# ==============================================
# 5. Train final model with best params and log to MLflow
# ==============================================
best_params = study.best_trial.params
best_model = XGBRegressor(**best_params)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_eval)

mae = mean_absolute_error(y_eval, y_pred)
rmse = np.sqrt(mean_squared_error(y_eval, y_pred))
r2 = r2_score(y_eval, y_pred)

print("Final tuned model performance:")
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)

# Log final model
with mlflow.start_run(run_name="best_xgboost_model"):
    mlflow.log_params(best_params)
    mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})
    mlflow.xgboost.log_model(best_model, name="model")

Final tuned model performance:
MAE: 33260.45015311102
RMSE: 69740.5388099546
R²: 0.962413608943458


2025/11/08 20:32:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
